In [28]:
import docplex
from docplex.mp.model import Model
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
n = 5
M = 3
bundles = np.array([[0,1,1], [1,0,1]])
values = [3, 10]
solution = []
#np.random.randint(0, 2, size=(18, 10)).tolist()
reg = LinearRegression().fit(bundles, values)
bundle = [1,1,1]
coef = reg.coef_
intercept = reg.intercept_
print(np.sum(bundle * coef) + intercept)
#reg.predict([[1,1,1]])


#values = np.random.randint(0, 100, size=(18)).tolist()
#bundles = np.random.randint(0, 2, size=(18, 10)).tolist()
#reg = LinearRegression().fit(bundles, values)
#reg.predict([[0,1,1,0,0,0,0,0,0,0]])[0]

6.5


In [31]:
def galo_bids_init(value_model, bidder_id, n, presampled_n, presampled_algorithm, ml_model):

    values = np.random.randint(0, 100, size=(10)).tolist()
    bundles = np.random.randint(0, 2, size=(10, 5)).tolist()
    #print(bundles)
    M = len(bundles[0])

    # bundles random 18 bundles of length 10
    reg = LinearRegression().fit(bundles, values)

    r_max = []
    count = 10
    bigM = 6000
    while count <= 30:
        reg = LinearRegression().fit(bundles, values)
        coef = reg.coef_
        intercept = reg.intercept_
        print(coef)
        print(intercept)
        x_vectors = []
        x_distances = []
        for bundle in range(len(bundles)):

            m = Model(name='galo')

            r = m.continuous_var(name='r')
            
            x = m.binary_var_list(range(M), name='x')
            
            b = [m.binary_var(name=f'b_{j}') for j in range(len(values))]

            constraints = []

            y_prediction = m.sum(x[k]*coef[k] for k in range(M)) + intercept

            #y_prediction = np.sum(x*coef) + intercept

            constraints.append(values[bundle] - y_prediction <= r)
            constraints.append(y_prediction - values[bundle] <= r)

            for j in range(len(values)):
                constraints.append(values[j] - y_prediction + bigM * b[j] >= r)
                constraints.append(y_prediction - values[j] + bigM * (1 - b[j]) >= r)

            m.add_constraints(constraints)
            
            m.maximize(r)

            sol = m.solve()

            try:
                print(sol.solve_status)
            except:
                print('No solution found')
                #iterate over constraints
                for i in m.iter_constraints():
                    print(i)
                print('#################')
            vector = np.array([x[k].solution_value for k in range(M)])
            x_vectors.append(vector)
            distance = np.sum(np.abs(np.sum(vector * coef) + intercept - values[bundle]))
            x_distances.append(distance)

        chosen_bundle = x_vectors[np.argmax(x_distances)]
        value = 10
        values.append(value)
        bundles.append(chosen_bundle.tolist())
        print(chosen_bundle)
        print(value)
        count+=1

    # D = np.array(bundles)
    # D = np.hstack((D, np.array(values).reshape(-1, 1)))
    # print(D)
    # return (D)
    return bundles

results = galo_bids_init(None, 0, 20, 10, 'unif', None)
#results

[-58.05       -41.4        -31.51666667 -26.09166667 -26.09166667]
177.75000000000006
JobSolveStatus.OPTIMAL_SOLUTION
JobSolveStatus.OPTIMAL_SOLUTION
No solution found
58.050x_0+41.400x_1+31.517x_2+26.092x_3+26.092x_4-166.750 <= r
-58.050x_0-41.400x_1-31.517x_2-26.092x_3-26.092x_4+166.750 <= r
58.050x_0+41.400x_1+31.517x_2+26.092x_3+26.092x_4+6000b_0-78.750 >= r
-58.050x_0-41.400x_1-31.517x_2-26.092x_3-26.092x_4-6000b_0+6078.750 >= r
58.050x_0+41.400x_1+31.517x_2+26.092x_3+26.092x_4+6000b_1-148.750 >= r
-58.050x_0-41.400x_1-31.517x_2-26.092x_3-26.092x_4-6000b_1+6148.750 >= r
58.050x_0+41.400x_1+31.517x_2+26.092x_3+26.092x_4+6000b_2-166.750 >= r
-58.050x_0-41.400x_1-31.517x_2-26.092x_3-26.092x_4-6000b_2+6166.750 >= r
58.050x_0+41.400x_1+31.517x_2+26.092x_3+26.092x_4+6000b_3-106.750 >= r
-58.050x_0-41.400x_1-31.517x_2-26.092x_3-26.092x_4-6000b_3+6106.750 >= r
58.050x_0+41.400x_1+31.517x_2+26.092x_3+26.092x_4+6000b_4-176.750 >= r
-58.050x_0-41.400x_1-31.517x_2-26.092x_3-26.092x_4-6000b_4+

DOcplexException: Model<galo> did not solve successfully

In [3]:
results

NameError: name 'results' is not defined